# Water consumption (Minimal testing notebook)

IMPORTANT NOTE: ALL DE INPUT DATA MUST BE STORED IN THE STORAGE BACKEND

- DOWNLOAD THE DATA FROM https://cloudlab.urv.cat/josep/geospatial_usecase AND UPLOAD IT TO THE STORAGE BACKEND WITH:

lithops storage put MDT/PNOA_MDT05_ETRS89_HU30_0891_LID.tif geospatial-usecase

lithops storage put MDT/PNOA_MDT05_ETRS89_HU30_0892_LID.tif geospatial-usecase

lithops storage put MDT/PNOA_MDT05_ETRS89_HU30_0912_LID.tif geospatial-usecase

lithops storage put MDT/PNOA_MDT05_ETRS89_HU30_0913_LID.tif geospatial-usecase

lithops storage put MDT/PNOA_MDT05_ETRS89_HU30_0933_LID.tif geospatial-usecase

lithops storage put MDT/PNOA_MDT05_ETRS89_HU30_0934_LID.tif geospatial-usecase

lithops storage put MDT/PNOA_MDT05_ETRS89_HU30_0954_LID.tif geospatial-usecase

lithops storage put MDT/PNOA_MDT05_ETRS89_HU30_0955_LID.tif geospatial-usecase

The current notebook computes an interpolation of temperatures in each pixel based on SIAM extracted data.

A ~/.lithops/config correctly configured located at home directory is needed in order to connect with IBM Cloud. More information at https://github.com/lithops-cloud/lithops/tree/master/config.

In [1]:
from collections import defaultdict
from cloudbutton_geospatial.io_utils.plot import plot_random_blocks, plot_results
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from shapely.geometry import Point, MultiPoint, box
from pprint import pprint
from functools import partial
from lithops import Storage
import io
import os
import lithops
import requests
import shutil
import numpy as np
import pandas as pd
import math
import rasterio
import random

## Global parameters

Area outside the processed tile that we want to consider for taking SIAM stations into account:

In [2]:
AREA_OF_INFLUENCE = 4000

Lithops Variables:

In [3]:
BUCKET = 'geospatial-usecase'
COMPUTE_BACKEND = 'ibm_cf'
STORAGE_BACKEND = 'ibm_cos'
RUNTIME = 'jsampe/lithops-ibmcf-geospatial-v38:06'

Split tile into SPLITS$^2$ chunks:

In [4]:
SPLITS = 3

Correlation coefficient between elevation and temperature:

In [5]:
r = -0.0056

Elevation to interpolate temperature:

In [6]:
zdet = 2000

Day of year to calculate solar irradiation:

In [7]:
DAY_OF_YEAR = 50

Object storage key prefix, to keep objects organized:

In [8]:
cloud_storage = Storage(backend=STORAGE_BACKEND)

## Data preparation

This section fetches and uploads to COS the metadata used in the workflow. It can be skipped if the data is already in COS.

### SIAM data

In [9]:
url = 'http://siam.imida.es/apex/f?p=101:47:493289053024037:CSV::::'
# url = 'http://siam.imida.es/apex/f?p=101:48:2555846978143339:CSV::::'
siam_data = requests.get(url)
with open('siam_data.csv', 'wb') as siam_data_file:
    siam_data_file.write(siam_data.content)

In [10]:
def guess_nearest(x, y, field, stations):
    '''
    Compute field value at a given x,y point by getting the value of the closest station 
    '''
    from shapely.ops import nearest_points
    stations_of_interest = stations[(stations[field] != '-') & ((stations['X'] != x) | (stations['Y'] != y))]
    points = MultiPoint(stations_of_interest.apply(lambda row: Point(row['X'], row['Y']), axis=1 ).array)
    nearest = nearest_points(Point(x,y), points)[1]
    val = stations_of_interest[(stations_of_interest['X'] == nearest.x) &
                                (stations_of_interest['Y'] == nearest.y)]

    return stations_of_interest[(stations_of_interest['X'] == nearest.x) &
                                (stations_of_interest['Y'] == nearest.y)][field].iloc[0] 

Append locations of SIAM stations to previously downloaded data and write results to a CSV:

In [11]:
columns = {
    'Estación': 'COD',
    'Tmed <br> (ºC)': 'temp',
    'Hrmed <br> (%)': 'hr',
    'Vvmed <br> (m/seg)': 'v',
    'Eti.': 'dir',
    'Radmed <br> (w/m2)': 'rad',
    'Dvmed <br>  (º)': 'dir_deg'
}

siam_data = pd.read_csv('siam_data.csv', encoding='iso-8859-1',
                        sep=';', decimal=',', thousands='.', na_values = '-')
siam_data = siam_data[columns.keys()].rename(columns=columns)
siam_locations = pd.read_csv('siam_locations.csv', encoding='iso-8859-1', sep=';', decimal = ',', thousands='.')
siam = pd.merge(siam_locations, siam_data, on='COD')
siam['tdet'] = siam['temp'] + r * (zdet - siam['Cota'].to_numpy())
siam = siam[['X', 'Y', 'Cota', 'temp', 'hr', 'tdet', 'v'] + list(columns.values())]
# Guess wind direction of undefined values
siam['dir_deg'] = siam.apply(lambda row: row['dir_deg'] 
                                     if not math.isnan(row['dir_deg'])
                                     else guess_nearest(row['X'], row['Y'], 'dir_deg', siam), axis=1)
# Guess radiation of undefined values
siam['rad'] = siam.apply(lambda row: row['rad'] 
                                     if not math.isnan(row['rad'])
                                     else guess_nearest(row['X'], row['Y'], 'rad', siam), axis=1)
siam.to_csv('siam.csv', index=False)

Upload the output CSV to COS:

In [12]:
with open('siam.csv', 'rb') as siam_out_file:
    cloud_storage.put_object(bucket=BUCKET, key='siam.csv', body=siam_out_file)

Upload the Sahape file

In [13]:
with open('shapefiles/shapefile.zip', 'rb') as shapefile:
    cloud_storage.put_object(bucket=BUCKET, key='shapefile.zip', body=shapefile)

## Serverless computation

Input MDT tiles to process:

In [14]:
tiles = [os.path.splitext(os.path.basename(key))[0]
         for key in cloud_storage.list_keys(bucket=BUCKET, prefix='MDT')]
tiles

['PNOA_MDT05_ETRS89_HU30_0891_LID',
 'PNOA_MDT05_ETRS89_HU30_0892_LID',
 'PNOA_MDT05_ETRS89_HU30_0912_LID',
 'PNOA_MDT05_ETRS89_HU30_0913_LID',
 'PNOA_MDT05_ETRS89_HU30_0933_LID',
 'PNOA_MDT05_ETRS89_HU30_0934_LID',
 'PNOA_MDT05_ETRS89_HU30_0954_LID',
 'PNOA_MDT05_ETRS89_HU30_0955_LID']

In [15]:
# DEBUG
#tiles = ['PNOA_MDT05_ETRS89_HU30_0891_LID','PNOA_MDT05_ETRS89_HU30_0892_LID']
#tiles

Compute solar irradiation given a day of year using GRASS libraries:

In [16]:
def compute_solar_irradiation(inputFile, outputFile, crs='32630'):
    # Define grass working set
    GRASS_GISDB = 'grassdata'
    GRASS_LOCATION = 'GEOPROCESSING'
    GRASS_MAPSET = 'PERMANENT'
    GRASS_ELEVATIONS_FILENAME = 'ELEVATIONS'
    

    os.environ['GRASSBIN'] = 'grass76'
    from grass_session import Session
    import grass.script as gscript
    from grass.pygrass.modules.shortcuts import general as g
    from grass.pygrass.modules.shortcuts import raster as r
    import re
    
    os.environ.update(dict(GRASS_COMPRESS_NULLS='1'))
    
    # Clean previously processed data
    if os.path.isdir(GRASS_GISDB):
        shutil.rmtree(GRASS_GISDB)
    with Session(gisdb=GRASS_GISDB, location=GRASS_LOCATION, mapset=GRASS_MAPSET, create_opts='EPSG:32630') as ses:
    
        # Set project projection to match elevation raster projection
        g.proj(epsg=crs, flags='c') 
    
        # Load raster file into working directory
        r.import_(input=inputFile, 
                  output=GRASS_ELEVATIONS_FILENAME, 
                  flags='o')    
        
        # Set project region to match raster region
        g.region(raster=GRASS_ELEVATIONS_FILENAME, flags='s')    
        # Calculate solar irradiation
        gscript.run_command('r.slope.aspect', elevation=GRASS_ELEVATIONS_FILENAME,
                            slope='slope', aspect='aspect')
        gscript.run_command('r.sun', elevation=GRASS_ELEVATIONS_FILENAME,
                            slope='slope', aspect='aspect', beam_rad='beam',
                            step=1, day=DAY_OF_YEAR)
        
        # Get extraterrestrial irradiation from history metadata
        regex = re.compile(r'\d+\.\d+')
        output = gscript.read_command("r.info", flags="h", map=["beam"])
        splits = str(output).split('\n')
        line = next(filter(lambda line: 'Extraterrestrial' in line, splits))
        extraterrestrial_irradiance = float(regex.search(line)[0])
        
        # Export generated results into a GeoTiff file
        if os.path.isfile(outputFile):
            os.remove(outputFile)
            
        r.out_gdal(input='beam', output=outputFile)
        
        return extraterrestrial_irradiance

Get stations contained in the area of interest:

In [17]:
def filter_stations(bounds, stations):
    total_points = MultiPoint([Point(x,y) for x, y in stations[['X', 'Y']].to_numpy()])
    intersection = bounds.buffer(AREA_OF_INFLUENCE).intersection(total_points)
    
    return stations[[ intersection.contains(point) for point in total_points]]

Inverse Distance Weighting interpolation:

In [18]:
def compute_basic_interpolation(shape, stations, field_value, offset = (0,0)):
    station_pixels = [[pixel[0], pixel[1]] for pixel in stations['pixel'].to_numpy()]
    
    # Get an array where each position represents pixel coordinates
    tile_pixels = np.indices(shape).transpose(1,2,0).reshape(shape[0]*shape[1], 2) + offset
    dist = distance_matrix(station_pixels, tile_pixels)
    weights = np.where(dist == 0, np.finfo('float32').max, 1.0 / dist )
    weights /=  weights.sum(axis=0)
    
    return np.dot(weights.T, stations[field_value].to_numpy()).reshape(shape).astype('float32')

Interpolate temperatures from a subset of the tile:

In [19]:
def radiation_interpolation(obj, block_x, block_y, splits, storage):
    tile_key = os.path.basename(obj.key)
    tile_id, _ = os.path.splitext(tile_key)
    
    with rasterio.open(obj.data_stream) as src:
        transform = src.transform
        
        # Compute working window
        step_w = src.width / splits
        step_h = src.height / splits
        
        offset_h = round(step_h * block_x)
        offset_w = round(step_w * block_y)
        
        profile = src.profile
        width = math.ceil(step_w * (block_y + 1) - offset_w)
        height = math.ceil(step_h * (block_x + 1) - offset_h)
        
        profile.update(width=width)
        profile.update(height=height)
        
        window = Window(offset_w, offset_h, width, height)
        
        with rasterio.open('input', 'w', **profile) as dest:
            dest.write(src.read(window=window))
        
    # Stores global irradiation at "output", it also returns extraterrestrial irradiation
    extraterrestrial_irradiation = compute_solar_irradiation('input', 'output')
        
    # Create and store a raster with extraterrestrial_irradiation
    with rasterio.open('extr', 'w', **profile) as dest:
        data = np.full((height, width), extraterrestrial_irradiation, dtype='float32')
        dest.write(data, 1)
        
    out_key = os.path.join('tmp', 'extrad', tile_id, f'chunk_{block_x}-{block_y}') + '.tif'
    with open('extr', 'rb') as out_file:
        storage.put_object(BUCKET, out_key, out_file)
    
    out_key = os.path.join('tmp', 'rad', tile_id, f'chunk_{block_x}-{block_y}') + '.tif'
    with open('output', 'rb') as out_file:
        storage.put_object(BUCKET, out_key, out_file)
    
    return out_key

In [20]:
def map_interpolation(obj, block_x, block_y, splits, data_field, storage):    
    tile_key = os.path.basename(obj.key)
    tile_id, _ = os.path.splitext(tile_key)
          
    siam_stream = storage.get_object(BUCKET, 'siam.csv', stream=True)
    siam = pd.read_csv(siam_stream)
    
    with rasterio.open(obj.data_stream) as src:
        transform = src.transform
        
        # Compute working window
        step_w = src.width / splits
        step_h = src.height / splits
        
        offset_h = round(step_h * block_x)
        offset_w = round(step_w * block_y)
        
        profile = src.profile
        
        width = math.ceil(step_w * (block_y + 1) - offset_w)
        height = math.ceil(step_h * (block_x + 1) - offset_h)
        
        profile.update(width=width)
        profile.update(height=height)
        
        window = Window(offset_w,offset_h, width, height)
        
        # Filter desired stations
        bounding_rect = box(src.bounds.left, src.bounds.top, src.bounds.right, src.bounds.bottom)
        filtered = pd.DataFrame(filter_stations(bounding_rect, siam))
        filtered['pixel'] = filtered.apply(
            lambda station: rasterio.transform.rowcol(transform, station['X'], station['Y']), axis=1)
        
        # Interpolate and write results 
        with rasterio.open('output', 'w', **profile) as dest:
            if data_field == 'temp':
                elevations = src.read(1, window=window) # Get elevations content
                interpolation = compute_basic_interpolation(elevations.shape, filtered,
                                                            'tdet', (offset_h, offset_w))
                interpolation += r * (elevations - zdet)
                dest.write(np.where(elevations == src.nodata, np.nan, interpolation), 1)
            else:
                interpolation = compute_basic_interpolation((height, width),
                                                            filtered, 
                                                            'hr' if data_field == 'humi' else 'v', 
                                                            (offset_h, offset_w))
                dest.write(interpolation, 1)

    # Export results to storage
    out_key = os.path.join('tmp', data_field, tile_id, 'chunk_{}-{}'.format(block_x, block_y)) + '.tif'
    with open('output', 'rb') as output_file:
        storage.put_object(BUCKET, out_key, output_file)
    
    return out_key

Lithops serverless computation:

In [21]:
iterdata = [('{}/MDT/{}.tif'.format(BUCKET, tile), i, j) 
            for i in range(SPLITS) for j in range(SPLITS) for tile in tiles]

In [22]:
pprint(iterdata)
print('Total functions: {} tiles * ({}^2) splits * 4 calculations = {}'.format(
    len(tiles), SPLITS, len(iterdata) * 4))

[('geospatial-usecase/MDT/PNOA_MDT05_ETRS89_HU30_0891_LID.tif', 0, 0),
 ('geospatial-usecase/MDT/PNOA_MDT05_ETRS89_HU30_0892_LID.tif', 0, 0),
 ('geospatial-usecase/MDT/PNOA_MDT05_ETRS89_HU30_0912_LID.tif', 0, 0),
 ('geospatial-usecase/MDT/PNOA_MDT05_ETRS89_HU30_0913_LID.tif', 0, 0),
 ('geospatial-usecase/MDT/PNOA_MDT05_ETRS89_HU30_0933_LID.tif', 0, 0),
 ('geospatial-usecase/MDT/PNOA_MDT05_ETRS89_HU30_0934_LID.tif', 0, 0),
 ('geospatial-usecase/MDT/PNOA_MDT05_ETRS89_HU30_0954_LID.tif', 0, 0),
 ('geospatial-usecase/MDT/PNOA_MDT05_ETRS89_HU30_0955_LID.tif', 0, 0),
 ('geospatial-usecase/MDT/PNOA_MDT05_ETRS89_HU30_0891_LID.tif', 0, 1),
 ('geospatial-usecase/MDT/PNOA_MDT05_ETRS89_HU30_0892_LID.tif', 0, 1),
 ('geospatial-usecase/MDT/PNOA_MDT05_ETRS89_HU30_0912_LID.tif', 0, 1),
 ('geospatial-usecase/MDT/PNOA_MDT05_ETRS89_HU30_0913_LID.tif', 0, 1),
 ('geospatial-usecase/MDT/PNOA_MDT05_ETRS89_HU30_0933_LID.tif', 0, 1),
 ('geospatial-usecase/MDT/PNOA_MDT05_ETRS89_HU30_0934_LID.tif', 0, 1),
 ('geo

In [23]:
fexec = lithops.FunctionExecutor(backend=COMPUTE_BACKEND, storage=STORAGE_BACKEND,
                                 runtime=RUNTIME, log_level="DEBUG")

fexec.map(radiation_interpolation, iterdata, extra_args=(SPLITS,), runtime_memory=2048)

fexec.map(map_interpolation, iterdata, extra_args=(SPLITS,'temp'), runtime_memory=1024)

fexec.map(map_interpolation, iterdata, extra_args=(SPLITS,'humi'), runtime_memory=1024)

fexec.map(map_interpolation, iterdata, extra_args=(SPLITS,'wind'), runtime_memory=1024)

out_chunks = fexec.get_result()

2021-09-21 22:43:20,667 [INFO] lithops.config -- Lithops v2.5.1.dev0
2021-09-21 22:43:20,668 [DEBUG] lithops.config -- Loading configuration from /home/josep/.lithops/config
2021-09-21 22:43:20,679 [DEBUG] lithops.config -- Loading Serverless backend module: ibm_cf
2021-09-21 22:43:20,682 [DEBUG] lithops.config -- Loading Storage backend module: ibm_cos
2021-09-21 22:43:20,682 [DEBUG] lithops.storage.backends.ibm_cos.ibm_cos -- Creating IBM COS client
2021-09-21 22:43:20,683 [DEBUG] lithops.storage.backends.ibm_cos.ibm_cos -- Set IBM COS Endpoint to https://s3.us-east.cloud-object-storage.appdomain.cloud
2021-09-21 22:43:20,684 [DEBUG] lithops.storage.backends.ibm_cos.ibm_cos -- Using access_key and secret_key
2021-09-21 22:43:20,689 [INFO] lithops.storage.backends.ibm_cos.ibm_cos -- IBM COS Storage client created - Region: us-east
2021-09-21 22:43:20,690 [DEBUG] lithops.serverless.backends.ibm_cf.ibm_cf -- Creating IBM Cloud Functions client
2021-09-21 22:43:20,691 [DEBUG] lithops.ser

2021-09-21 22:43:34,984 [DEBUG] lithops.job.partitioner -- Creating 1 partitions from object MDT/PNOA_MDT05_ETRS89_HU30_0912_LID.tif (61.9MiB)
2021-09-21 22:43:34,984 [DEBUG] lithops.job.partitioner -- Creating 1 partitions from object MDT/PNOA_MDT05_ETRS89_HU30_0913_LID.tif (56.3MiB)
2021-09-21 22:43:34,985 [DEBUG] lithops.job.partitioner -- Creating 1 partitions from object MDT/PNOA_MDT05_ETRS89_HU30_0933_LID.tif (62.2MiB)
2021-09-21 22:43:34,985 [DEBUG] lithops.job.partitioner -- Creating 1 partitions from object MDT/PNOA_MDT05_ETRS89_HU30_0934_LID.tif (60.1MiB)
2021-09-21 22:43:34,987 [DEBUG] lithops.job.partitioner -- Creating 1 partitions from object MDT/PNOA_MDT05_ETRS89_HU30_0954_LID.tif (57.3MiB)
2021-09-21 22:43:34,988 [DEBUG] lithops.job.partitioner -- Creating 1 partitions from object MDT/PNOA_MDT05_ETRS89_HU30_0955_LID.tif (53.6MiB)
2021-09-21 22:43:34,988 [DEBUG] lithops.job.partitioner -- Creating 1 partitions from object MDT/PNOA_MDT05_ETRS89_HU30_0891_LID.tif (62.6MiB)

2021-09-21 22:43:35,394 [INFO] lithops.invokers -- ExecutorID dd8057-0 | JobID M000 - View execution logs at /tmp/lithops/logs/dd8057-0-M000.log
2021-09-21 22:43:35,395 [DEBUG] lithops.monitor -- ExecutorID dd8057-0 - Starting Storage job monitor
2021-09-21 22:43:35,511 [INFO] lithops.invokers -- ExecutorID dd8057-0 | JobID M001 - Selected Runtime: jsampe/lithops-ibmcf-geospatial-v38:06 - 1024MB
2021-09-21 22:43:35,521 [DEBUG] lithops.storage.storage -- Runtime metadata not found in local cache. Retrieving it from storage
2021-09-21 22:43:35,522 [DEBUG] lithops.storage.storage -- Trying to download runtime metadata from: ibm_cos://lithops-data-us-east/lithops.runtimes/2.5.1.dev0/ibm_cf/us-east/josep_dev_us_east/jsampe_lithops-ibmcf-geospatial-v38_06_1024MB.meta.json
2021-09-21 22:43:35,745 [DEBUG] lithops.storage.storage -- Runtime metadata not found in storage
2021-09-21 22:43:35,746 [INFO] lithops.invokers -- Runtime jsampe/lithops-ibmcf-geospatial-v38:06 with 1024MB is not yet insta

2021-09-21 22:43:35,887 [DEBUG] lithops.invokers -- ExecutorID dd8057-0 | JobID M000 - Calls 00035 invoked (0.511s) - Activation ID: 75d377263903430f9377263903930f6a
2021-09-21 22:43:35,887 [DEBUG] lithops.invokers -- ExecutorID dd8057-0 | JobID M000 - Calls 00032 invoked (0.512s) - Activation ID: 0aa6b0d444ad4727a6b0d444adc72796
2021-09-21 22:43:35,888 [DEBUG] lithops.invokers -- ExecutorID dd8057-0 | JobID M000 - Calls 00047 invoked (0.506s) - Activation ID: de3968a1e9564dfab968a1e956ddfa97
2021-09-21 22:43:35,889 [DEBUG] lithops.invokers -- ExecutorID dd8057-0 | JobID M000 - Calls 00062 invoked (0.500s) - Activation ID: ae4014b166a84e228014b166a89e2281
2021-09-21 22:43:35,889 [DEBUG] lithops.invokers -- ExecutorID dd8057-0 | JobID M000 - Calls 00020 invoked (0.521s) - Activation ID: 325ef3c0b6544a259ef3c0b6547a252f
2021-09-21 22:43:35,889 [DEBUG] lithops.invokers -- ExecutorID dd8057-0 | JobID M000 - Calls 00034 invoked (0.514s) - Activation ID: 20d22b73dbf941d7922b73dbf9d1d71f
2021

2021-09-21 22:44:09,018 [DEBUG] lithops.job.partitioner -- Listing objects in 'ibm_cos://geospatial-usecase/MDT'
2021-09-21 22:44:09,235 [DEBUG] lithops.job.partitioner -- Total objects found: 8
2021-09-21 22:44:09,236 [DEBUG] lithops.job.partitioner -- Creating 1 partitions from object MDT/PNOA_MDT05_ETRS89_HU30_0891_LID.tif (62.6MiB)
2021-09-21 22:44:09,237 [DEBUG] lithops.job.partitioner -- Creating 1 partitions from object MDT/PNOA_MDT05_ETRS89_HU30_0892_LID.tif (63.1MiB)
2021-09-21 22:44:09,238 [DEBUG] lithops.job.partitioner -- Creating 1 partitions from object MDT/PNOA_MDT05_ETRS89_HU30_0912_LID.tif (61.9MiB)
2021-09-21 22:44:09,239 [DEBUG] lithops.job.partitioner -- Creating 1 partitions from object MDT/PNOA_MDT05_ETRS89_HU30_0913_LID.tif (56.3MiB)
2021-09-21 22:44:09,240 [DEBUG] lithops.job.partitioner -- Creating 1 partitions from object MDT/PNOA_MDT05_ETRS89_HU30_0933_LID.tif (62.2MiB)
2021-09-21 22:44:09,241 [DEBUG] lithops.job.partitioner -- Creating 1 partitions from obje

...

2021-09-21 22:45:08,357 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M000 - Got output from call 00039 - Activation ID: fd350fb2f0984245b50fb2f098224591
2021-09-21 22:45:08,378 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M001 - Got status from call 00030 - Activation ID: 08c2dfe6eccd4ba082dfe6eccd9ba0d0 - Time: 12.73 seconds
2021-09-21 22:45:08,379 [DEBUG] lithops.monitor -- ExecutorID dd8057-0 - Pending: 8 - Running: 73 - Done: 207
2021-09-21 22:45:08,379 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M001 - Got status from call 00009 - Activation ID: d42d7c0588fd4ee2ad7c0588fd0ee245 - Time: 12.20 seconds
2021-09-21 22:45:08,380 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M001 - Got status from call 00008 - Activation ID: 99d65f46a44246e2965f46a442c6e23f - Time: 12.92 seconds
2021-09-21 22:45:08,515 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M001 - Got output from call 00030 - Activation ID: 08c2dfe6eccd4ba082dfe6eccd9ba0d0
2021-09-21 22:

2021-09-21 22:45:30,980 [DEBUG] lithops.monitor -- ExecutorID dd8057-0 - Pending: 8 - Running: 56 - Done: 224
2021-09-21 22:45:31,771 [DEBUG] lithops.monitor -- ExecutorID dd8057-0 - Pending: 8 - Running: 56 - Done: 224
2021-09-21 22:45:31,977 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M000 - Got status from call 00008 - Activation ID: 4127eaf6f4ed48a5a7eaf6f4ed68a530 - Time: 81.24 seconds
2021-09-21 22:45:31,978 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M000 - Got status from call 00058 - Activation ID: e2c6b0b2683646a886b0b2683636a825 - Time: 83.89 seconds
2021-09-21 22:45:32,140 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M000 - Got output from call 00008 - Activation ID: 4127eaf6f4ed48a5a7eaf6f4ed68a530
2021-09-21 22:45:32,154 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M000 - Got output from call 00058 - Activation ID: e2c6b0b2683646a886b0b2683636a825
2021-09-21 22:45:34,183 [DEBUG] lithops.monitor -- ExecutorID dd8057-0 - Pending: 8 - Ru

2021-09-21 22:46:01,973 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M000 - Got output from call 00057 - Activation ID: 45cb08e2f6c04b158b08e2f6c0bb1547
2021-09-21 22:46:02,895 [DEBUG] lithops.monitor -- ExecutorID dd8057-0 - Pending: 0 - Running: 47 - Done: 241
2021-09-21 22:46:02,978 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M000 - Got status from call 00029 - Activation ID: dbb2ac1db04e484db2ac1db04e684df1 - Time: 113.35 seconds
2021-09-21 22:46:03,136 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M000 - Got output from call 00029 - Activation ID: dbb2ac1db04e484db2ac1db04e684df1
2021-09-21 22:46:03,835 [DEBUG] lithops.monitor -- ExecutorID dd8057-0 - Pending: 0 - Running: 47 - Done: 241
2021-09-21 22:46:06,395 [DEBUG] lithops.monitor -- ExecutorID dd8057-0 - Pending: 0 - Running: 45 - Done: 243
2021-09-21 22:46:07,145 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M002 - Got status from call 00009 - Activation ID: 065754783e15409a9754783e15809a0f

2021-09-21 22:46:23,323 [DEBUG] lithops.monitor -- ExecutorID dd8057-0 - Pending: 0 - Running: 28 - Done: 260
2021-09-21 22:46:23,927 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M000 - Got status from call 00067 - Activation ID: 2c71d8dd5e114a06b1d8dd5e117a067b - Time: 125.09 seconds
2021-09-21 22:46:24,064 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M000 - Got output from call 00067 - Activation ID: 2c71d8dd5e114a06b1d8dd5e117a067b
2021-09-21 22:46:24,765 [DEBUG] lithops.monitor -- ExecutorID dd8057-0 - Pending: 0 - Running: 28 - Done: 260
2021-09-21 22:46:27,261 [DEBUG] lithops.monitor -- ExecutorID dd8057-0 - Pending: 0 - Running: 27 - Done: 261
2021-09-21 22:46:28,076 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M000 - Got status from call 00003 - Activation ID: f6708adcf48f4409b08adcf48f64097e - Time: 151.24 seconds
2021-09-21 22:46:28,232 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M000 - Got output from call 00003 - Activation ID: f6708adcf

2021-09-21 22:47:08,051 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M000 - Got output from call 00009 - Activation ID: 7457381b8c8b466797381b8c8b766725
2021-09-21 22:47:08,447 [DEBUG] lithops.monitor -- ExecutorID dd8057-0 - Pending: 0 - Running: 12 - Done: 276
2021-09-21 22:47:11,069 [DEBUG] lithops.monitor -- ExecutorID dd8057-0 - Pending: 0 - Running: 12 - Done: 276
2021-09-21 22:47:13,955 [DEBUG] lithops.monitor -- ExecutorID dd8057-0 - Pending: 0 - Running: 11 - Done: 277
2021-09-21 22:47:14,066 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M000 - Got status from call 00060 - Activation ID: 5e794f47ffae4b01b94f47ffae7b01d3 - Time: 185.43 seconds
2021-09-21 22:47:14,237 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M000 - Got output from call 00060 - Activation ID: 5e794f47ffae4b01b94f47ffae7b01d3
2021-09-21 22:47:15,049 [DEBUG] lithops.monitor -- ExecutorID dd8057-0 - Pending: 0 - Running: 10 - Done: 278
2021-09-21 22:47:15,242 [DEBUG] lithops.future -- E

Join split subsets into a tile:

In [24]:
def gather_blocks(tile, splits, data_field, storage):

    from rasterio.windows import Window
    
    # Get width and height from original tile
    with rasterio.open(storage.get_object(bucket=BUCKET, key=f'MDT/{tile}.tif', stream=True)) as og:
        height = og.profile['height']
        width = og.profile['width']
    
    chunk_tiles = storage.list_keys(bucket=BUCKET, prefix=f'tmp/{data_field}/{tile}/chunk')
        
    # Open first object to obtain profile metadata
    with rasterio.open(storage.get_object(bucket=BUCKET, key=chunk_tiles[0], stream=True)) as src:
        profile = src.profile
        profile.update(width=width)
        profile.update(height=height)

    # Iterate each object and print its block into the destination file
    with rasterio.open("output", "w", **profile) as dest: 
        for chunk in chunk_tiles:
            j, i = os.path.splitext(os.path.basename(chunk))[0].rsplit('_')[1].split('-')
            j, i = int(j), int(i)
            with rasterio.open(storage.get_object(bucket=BUCKET, key=chunk, stream=True)) as src:
                step_w = math.floor(width / splits)
                step_h = math.floor(height / splits)
                curr_window = Window(round(step_w * i), round(step_h * j), src.width, src.height)
                content = src.read(1)
                dest.write(content, 1, window=curr_window)
            # storage.delete_object(bucket=BUCKET, key=chunk)
    
    output_key = os.path.join('tmp', data_field, tile, '_'.join([tile, data_field.upper()+'.tif']))
    with open('output', 'rb') as out_file:
        storage.put_object(bucket=BUCKET, key=output_key, body=out_file)  
    
    return output_key

Combine previous split subsets:

In [25]:
fexec.map(gather_blocks, tiles, extra_args=(SPLITS, 'extrad'), runtime_memory=1024)

fexec.map(gather_blocks, tiles, extra_args=(SPLITS, 'humi'), runtime_memory=1024)

fexec.map(gather_blocks, tiles, extra_args=(SPLITS, 'rad'), runtime_memory=1024)

fexec.map(gather_blocks, tiles, extra_args=(SPLITS, 'temp'), runtime_memory=1024)

fexec.map(gather_blocks, tiles, extra_args=(SPLITS, 'wind'), runtime_memory=1024)

out_combined = fexec.get_result()

2021-09-21 22:48:20,119 [INFO] lithops.invokers -- ExecutorID dd8057-0 | JobID M004 - Selected Runtime: jsampe/lithops-ibmcf-geospatial-v38:06 - 1024MB
2021-09-21 22:48:20,120 [DEBUG] lithops.storage.storage -- Runtime metadata found in local memory cache
2021-09-21 22:48:20,121 [DEBUG] lithops.job.job -- ExecutorID dd8057-0 | JobID M004 - Serializing function and data
2021-09-21 22:48:20,126 [DEBUG] lithops.job.serialize -- Referenced modules: /usr/local/lib/python3.8/dist-packages/rasterio/__init__.py, /usr/lib/python3.8/os.py, /usr/local/lib/python3.8/dist-packages/rasterio/windows.py, math
2021-09-21 22:48:20,131 [DEBUG] lithops.job.serialize -- Modules to transmit: None
2021-09-21 22:48:20,132 [DEBUG] lithops.job.job -- ExecutorID dd8057-0 | JobID M004 - Uploading function and modules to the storage backend
2021-09-21 22:48:20,411 [DEBUG] lithops.storage.backends.ibm_cos.ibm_cos -- PUT Object lithops.jobs/dd8057-0/307abea834b66d449e9d45b05cbfd080.func.pickle - Size: 1.6KiB - OK
20

2021-09-21 22:48:20,919 [DEBUG] lithops.invokers -- ExecutorID dd8057-0 | JobID M004 - Calls 00005 invoked (0.491s) - Activation ID: 8376f98edba44b42b6f98edba4cb429c
2021-09-21 22:48:20,922 [DEBUG] lithops.invokers -- ExecutorID dd8057-0 | JobID M004 - Calls 00004 invoked (0.495s) - Activation ID: 5e15745ef78f477e95745ef78f177e2b
2021-09-21 22:48:20,926 [DEBUG] lithops.invokers -- ExecutorID dd8057-0 | JobID M004 - Calls 00000 invoked (0.505s) - Activation ID: 37470ee9cbbd48f7870ee9cbbde8f73e
2021-09-21 22:48:20,928 [DEBUG] lithops.invokers -- ExecutorID dd8057-0 | JobID M004 - Calls 00002 invoked (0.506s) - Activation ID: bd195321b9be43be995321b9be23bec5
2021-09-21 22:48:20,929 [DEBUG] lithops.invokers -- ExecutorID dd8057-0 | JobID M004 - Calls 00003 invoked (0.503s) - Activation ID: f4b5200ee1994035b5200ee199503556
2021-09-21 22:48:20,930 [DEBUG] lithops.invokers -- ExecutorID dd8057-0 | JobID M004 - Calls 00007 invoked (0.500s) - Activation ID: d4c57636ea7d4ad2857636ea7d7ad23d
2021

2021-09-21 22:48:28,551 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M006 - Got status from call 00001 - Activation ID: 4a378d56b62742a0b78d56b627b2a05f - Time: 4.98 seconds
2021-09-21 22:48:28,552 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M006 - Got status from call 00000 - Activation ID: a6b3e8af189f42abb3e8af189f22ab19 - Time: 5.78 seconds
2021-09-21 22:48:28,552 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M004 - Got status from call 00007 - Activation ID: d4c57636ea7d4ad2857636ea7d7ad23d - Time: 5.55 seconds
2021-09-21 22:48:28,555 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M006 - Got status from call 00003 - Activation ID: f68864b103e6481c8864b103e6581c2c - Time: 5.51 seconds
2021-09-21 22:48:28,548 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M004 - Got status from call 00005 - Activation ID: 8376f98edba44b42b6f98edba4cb429c - Time: 5.16 seconds
2021-09-21 22:48:28,688 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M004 - 

2021-09-21 22:48:58,844 [DEBUG] lithops.monitor -- ExecutorID dd8057-0 - Pending: 0 - Running: 17 - Done: 23
2021-09-21 22:48:59,502 [DEBUG] lithops.monitor -- ExecutorID dd8057-0 - Pending: 0 - Running: 17 - Done: 23
2021-09-21 22:48:59,838 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M005 - Got status from call 00002 - Activation ID: 746b9bea2bd34c25ab9bea2bd35c255d - Time: 36.79 seconds
2021-09-21 22:48:59,975 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M005 - Got output from call 00002 - Activation ID: 746b9bea2bd34c25ab9bea2bd35c255d
2021-09-21 22:49:01,985 [DEBUG] lithops.monitor -- ExecutorID dd8057-0 - Pending: 0 - Running: 15 - Done: 25
2021-09-21 22:49:01,990 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M005 - Got status from call 00005 - Activation ID: 1144641ce1724fb184641ce172bfb1ee - Time: 38.85 seconds
2021-09-21 22:49:01,991 [DEBUG] lithops.future -- ExecutorID dd8057-0 | JobID M005 - Got status from call 00007 - Activation ID: 5175c6d9a2294c

## Computation of potential evaporation

In [26]:
def compute_crop_evapotranspiration(temperatures,
                                    humidities,
                                    wind_speeds,
                                    external_radiations,
                                    global_radiations,
                                    KCs):
    gamma = 0.665*101.3/1000
    eSat = 0.6108 * np.exp((17.27*temperatures)/(temperatures+237.3))
    delta = 4098 * eSat / np.power((temperatures + 237.3),2)
    eA = np.where(humidities < 0, 0, eSat * humidities / 100)     # Avoid sqrt of a negative number
    T4 = 4.903 * np.power((273.3 + temperatures),4)/1000000000
    rSrS0 = global_radiations/(external_radiations * 0.75)
    rN = 0.8* global_radiations-T4*(0.34-0.14*np.sqrt(eA))*((1.35*rSrS0)-0.35)
    den = delta + gamma *(1 + 0.34* wind_speeds)
    tRad = 0.408 * delta * rN / den
    tAdv = gamma * (900/(temperatures+273))*wind_speeds * (eSat - eA)/den
    return ((tRad + tAdv) * 7 * KCs).astype('float32')

In [27]:
vineyard = ['VI', 'VO', 'VF', 'FV', 'CV' ]
olive_grove = ['OV', 'VO', 'OF', 'FL', 'OC']
fruit = ['FY', 'VF', 'OF', 'FF', 'CF']
nuts = ['FS', 'FV', 'FL', 'FF', 'CS' ]
citrus = ['CI', 'CV', 'OC', 'CF', 'CS' ]

def get_kc(feature):
    
    # TODO: Get more precise values of Kc
    sigpac_use = feature['properties']['uso_sigpac']
    if sigpac_use in vineyard:
        # Grapes for wine - 0.3, 0.7, 0.45
        return 0.7  
    if sigpac_use in olive_grove:
        # Olive grove - ini: 0.65, med: 0.7, end: 0.7
        return 0.7 
    if sigpac_use in fruit:
        # Apples, Cherries, Pears - 0.45, 0.95, 0.7
        return 0.95
    if sigpac_use in nuts:
        # Almonds - 0.4, 0.9, 0.65
        return 0.9
    if sigpac_use in citrus:
        # Citrus, without ground coverage - 0.7, 0.65, 0.7
        return 0.65
    
    return None

In [28]:
def get_geometry_window(src, geom_bounds):
    left, bottom, right, top = geom_bounds
    src_left, src_bottom, src_right, src_top = src.bounds
    window = src.window(max(left,src_left), max(bottom,src_bottom), min(right,src_right), min(top,src_top))
    window_floored = window.round_offsets(op='floor', pixel_precision=3)
    w = math.ceil(window.width + window.col_off - window_floored.col_off)
    h = math.ceil(window.height + window.row_off - window_floored.row_off)
    return Window(window_floored.col_off, window_floored.row_off, w, h)     

In [29]:
def compute_evapotranspiration_by_shape(tem, hum, win, rad, extrad, dst):
    
    import fiona
    from shapely.geometry import shape, box
    from rasterio import features
    
    non_arable_land = ['AG', 'CA', 'ED', 'FO', 'IM', 'PA', 'PR', 'ZU', 'ZV']
    
    with fiona.open('zip://shape.zip') as shape_src:
        for feature in shape_src.filter(bbox=tem.bounds):
            KC = get_kc(feature) 
            if KC is not None:   
                geom = shape(feature['geometry'])  
                window = get_geometry_window(tem, geom.bounds)              
                win_transform = rasterio.windows.transform(window, tem.transform)
                # Convert shape to raster matrix
                image = features.rasterize([geom],
                                           out_shape=(window.height, window.width),
                                           transform = win_transform,
                                           fill = 0,
                                           default_value = 1).astype('bool')
                # Get values to compute evapotranspiration
                temperatures = tem.read(1, window=window)
                humidities = hum.read(1, window=window)
                wind_speeds = win.read(1, window=window)
                # Convert from W to MJ (0.0036)
                global_radiations = rad.read(1, window=window) * 0.0036
                external_radiations = extrad.read(1, window=window) * 0.0036
                KCs = np.full(temperatures.shape, KC)
                # TODO: compute external radiation
                #external_radiations = np.full(temperatures.shape, 14)
                # TODO: compute global radiation
                # global_radiations = np.full(temperatures.shape, 10)
                etc = compute_crop_evapotranspiration(
                        temperatures,
                        humidities,
                        wind_speeds,
                        external_radiations,
                        global_radiations,
                        KCs
                )
                etc[temperatures == tem.nodata] = dst.nodata
                etc[np.logical_not(image)] = dst.nodata
                dst.write(etc + dst.read(1, window=window), 1, window=window)

In [30]:
def compute_global_evapotranspiration(tem, hum, win, rad, extrad, dst):    
    for ji, window in tem.block_windows(1):
        bounds = rasterio.windows.bounds(window, tem.transform)
        temperatures = tem.read(1, window=window)
        humidities = hum.read(1, window=window)
        wind_speeds = win.read(1, window=window)
         # Convert from W to MJ (0.0036)
        global_radiations = rad.read(1, window=window) * 0.0036
        external_radiations = extrad.read(1, window=window) * 0.0036
        # TODO: compute external radiation
        #external_radiations = np.full(temperatures.shape, 14)
        # TODO: compute global radiation
        # global_radiations = np.full(temperatures.shape, 10)
        # TODO: compute KCs
        KCs = np.full(temperatures.shape, 1)
        etc = compute_crop_evapotranspiration(
                temperatures,
                humidities,
                wind_speeds,
                external_radiations,
                global_radiations,
                KCs
        )
        dst.write(np.where(temperatures == tem.nodata, dst.nodata, etc), 1, window=window)

In [31]:
def combine_calculations(tile, storage):      
    # Download shapefile
    shapefile = storage.get_object(bucket=BUCKET, key='shapefile.zip', stream=True)
    with open('shape.zip', 'wb') as shapf:
        for chunk in iter(partial(shapefile.read, 200 * 1024 * 1024), ''):
            if not chunk:
                break
            shapf.write(chunk)
    
    temp = storage.get_object(bucket=BUCKET, key=f'tmp/temp/{tile}/{tile}_TEMP.tif', stream=True)
    humi = storage.get_object(bucket=BUCKET, key=f'tmp/humi/{tile}/{tile}_HUMI.tif', stream=True)
    rad = storage.get_object(bucket=BUCKET, key=f'tmp/rad/{tile}/{tile}_RAD.tif', stream=True)
    extrad = storage.get_object(bucket=BUCKET, key=f'tmp/extrad/{tile}/{tile}_EXTRAD.tif', stream=True)
    wind = storage.get_object(bucket=BUCKET, key=f'tmp/wind/{tile}/{tile}_WIND.tif', stream=True)
    
    with rasterio.open(temp) as temp_raster:
        with rasterio.open(humi) as humi_raster:
            with rasterio.open(rad) as rad_raster:
                with rasterio.open(extrad) as extrad_raster:
                    with rasterio.open(wind) as wind_raster:
                        profile = temp_raster.profile
                        profile.update(nodata=0)
        
                        with rasterio.open('output', 'w+', **profile) as dst:
#                             compute_global_evapotranspiration(temp_raster, humi_raster, wind_raster,
#                                                               rad_raster, extrad_raster, dst)
                            compute_evapotranspiration_by_shape(temp_raster, humi_raster, wind_raster,
                                                                rad_raster, extrad_raster, dst)
    
    out_key = f'etc/{tile}_ETC.tif'
    with open('output', 'rb') as output_f:
        storage.put_object(bucket=BUCKET, key=out_key, body=output_f)
    return out_key

In [32]:
fexec.map(combine_calculations, tiles, runtime_memory=2048)

res = fexec.get_result()

2021-09-21 22:49:12,369 [INFO] lithops.invokers -- ExecutorID dd8057-0 | JobID M009 - Selected Runtime: jsampe/lithops-ibmcf-geospatial-v38:06 - 2048MB
2021-09-21 22:49:12,371 [DEBUG] lithops.storage.storage -- Runtime metadata found in local disk cache
2021-09-21 22:49:12,372 [DEBUG] lithops.job.job -- ExecutorID dd8057-0 | JobID M009 - Serializing function and data
2021-09-21 22:49:12,384 [DEBUG] lithops.job.serialize -- Referenced modules: /usr/local/lib/python3.8/dist-packages/rasterio/__init__.py, /usr/local/lib/python3.8/dist-packages/shapely/geometry/geo.py, /usr/local/lib/python3.8/dist-packages/shapely/geometry/polygon.py, /home/josep/.local/lib/python3.8/site-packages/numpy/__init__.py, /usr/local/lib/python3.8/dist-packages/shapely/__init__.py, /usr/lib/python3.8/functools.py, /usr/local/lib/python3.8/dist-packages/shapely/geometry/__init__.py, math, /usr/local/lib/python3.8/dist-packages/rasterio/windows.py
2021-09-21 22:49:12,384 [DEBUG] lithops.job.serialize -- Modules to

KeyError: 'uso_sigpac'

## Visualization of results

In [ ]:
import io
tile = random.choice(tiles)
obj = io.BytesIO(cloud_storage.get_object(bucket=BUCKET, key=f'etc/{tile}_ETC.tif'))

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots()

with rasterio.open(obj) as src:
    arr = src.read(1, out_shape=(src.height, src.width))
    ax.set_title(tile)
    img = ax.imshow(arr, cmap='Greens')
    fig.colorbar(img, shrink=0.5)

fig.set_size_inches(18.5, 10.5)
plt.show()

obj.seek(0)

### Utility functions

Remove intermediate data

In [ ]:
# keys = cloud_storage.list_keys(bucket=BUCKET, prefix='')
# keys

In [ ]:
# for key in keys:
#     cloud_storage.delete_object(bucket=BUCKET, key=key)